In [1]:
%load_ext google.cloud.bigquery
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


This will select data from BQ into a dataframe

In [ ]:
%%bigquery prices
SELECT * FROM `kaggle-covid.sell_prices.sell_prices`

In [ ]:
prices

In [ ]:
%%bigquery calendar
SELECT * FROM `kaggle-covid.calendar.calendar`;


In [ ]:
calendar

In [ ]:
%%bigquery item_013
-- select one item in one store
select 
*
from
sales_train_evaluation.sales_train_evaluation a
where a.store_id = 'CA_1' and a.item_id = 'HOBBIES_1_013';

In [ ]:
item_013

In [ ]:
%%bigquery store_CA_1
-- select all items from one store
select 
id, a.item_id, dept_id,cat_id, a.store_id, state_id, d_1, d_1941
from
sales_train_evaluation.sales_train_evaluation a
where a.store_id = 'CA_1';

In [ ]:
store_CA_1

Pivot all the date columns into rows to get something more amenable for time series prediction 

In [ ]:
df = pd.melt(
    store_CA_1,
    id_vars=[
        'id',
        'item_id',
        'dept_id',
        'cat_id',
        'store_id',
        'state_id'
    ],
    var_name='d',
    value_name='items_sold'
).dropna()

In [ ]:
df

Join with calendar dataframe

In [ ]:
df = pd.merge(df, calendar, on='d', how='left')
df

Now join with item prices for each item within store for that date (wm_yrwk).

In [ ]:
df = pd.merge(
    df,
    prices,
    on=['store_id','item_id','wm_yr_wk'],
    how='left') 

In [ ]:
df.describe

In [ ]:
df.head()

In [50]:
df.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            6088, 6089, 6090, 6091, 6092, 6093, 6094, 6095, 6096, 6097],
           dtype='int64', length=6098)

In [51]:
df.loc[df['wm_yr_wk'] == 11101]

,id,item_id,dept_id,cat_id,store_id,state_id,d,items_sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
3,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
4,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,FOODS_3_319_CA_1_evaluation,FOODS_3_319,FOODS_3,FOODS,CA_1,CA,d_1,5,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,1.00
3045,FOODS_3_681_CA_1_evaluation,FOODS_3_681,FOODS_3,FOODS,CA_1,CA,d_1,12,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,1.25
3046,FOODS_3_080_CA_1_evaluation,FOODS_3_080,FOODS_3,FOODS,CA_1,CA,d_1,33,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,1.48
3047,FOODS_3_586_CA_1_evaluation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_1,42,2011-01-29,11101,...,1,2011,None,None,None,None,0,0,0,1.48


In [ ]:
def plot_values(df, from_date=None, target=None):
    fig = px.bar(df.loc[df['date'] >= from_date], x='date', \
             y='items_sold', color="items_sold", width=800, height=400, color_continuous_scale=px.colors.sequential.BuGn)
    fig.show()
    
plot_values(df, '2011-01-29')